# Magnetic Anisotropy

In Chapter 4 of *Essentials of Paleomagnetism*, Lisa Tauxe introduces the concept of **Magnetic Anisotropy**—the phenomena that magnetic moments have a preferred direction within a crystal. This preference is what allows assemblages of magnetic minerals to record Earth's magnetic field.

**Class Structure:**
In this session, we will explore the competition between different forms of magnetic energy focusing on the mineral magnetite:
1.  **Magnetocrystalline Anisotropy:** The barriers to the rotation of magnetization as a resulted of the crystal lattice. We will focus on magnetite.
2.  **Shape Anisotropy:** How the elongated shape of a grain can creates result in strong anisotropy.
3.  **The "Tug of War":** Determining how much a grain needs to be elongated for shape anisotropy to overcome magnetocrystalline anisotropy.
4.  **Time & Temperature:** Determining the smallest grain that can hold a record for a billion years.

**Homework Assignment:**
At the end of this notebook, you will apply these concepts to answer a specific research question:
> **"What are the smallest magnetite grains that could reliably record a magnetic signal for a billion years (i.e., since the Mesoproterozoic)?"**

## Why is anisotropy important?

> Anisotropy energy creates barriers to free rotation of the magnetization within the magnetic crystal, which lead to energetically preferred directions for the magnetization within individual single-domain grains.
There are many causes of anisotropy energy. The most important ones derive from
the details of crystal structure (magnetocrystalline anisotropy energy), the state of stress within the particle (magnetostriction), and the shape of the particle (shape anisotropy).  *Tauxe (2010)*

Through a magnetic grain seeking to minimize its energy it naturally aligns with specific "easy" directions. This directional preference creates stability, preventing the magnetization from drifting freely and is therefore central to magnetic mineral assemblages being magnetic recorders.

## Import tools we are going to need

In [ ]:
import plotly.graph_objects as go
import numpy as np
import itertools
import matplotlib.pyplot as plt  # For plotting graphs
from ipywidgets import interact, FloatSlider, IntSlider # For interactivity
import ipywidgets as widgets
from IPython.display import display
from matplotlib.patches import Ellipse


## Magnetite Crystal Structure and Crystallographic Axes

Magnetite (Fe₃O₄) crystallizes in the **cubic inverse-spinel structure**, part of the isometric (cubic) crystal system.

<figure>
  <img src="https://raw.githubusercontent.com/Institute-for-Rock-Magnetism/2026_ESCI_pmag_course/2984338b2741d475d8f100893501381145ad9577/W3_anisotropy_hysteresis/images/magnetite.png" width="800px">
  <figcaption>
    <em>
      a) A magnetite octahedron. [Photo by Lou Perloff in the Photo-Atlas of Minerals.] b) Internal
crystal structure. Directions of the body diagonal ([111] direction) and orthogonal to the cubic faces ([001]
direction) are shown as arrows. Big red dots are the oxygen anions. The blue dots are iron cations in octahedral
coordination, and the yellow dots are in tetrahedral coordination. Fe3+ sits on the A sites, and Fe2+ and Fe3+
sit on the B sites. c) Magnetocrystalline anisotropy energy as a function of direction within a magnetite crystal
at room temperature. The easiest direction to magnetize (the direction with the lowest energy—note dimples in
energy surface) is along the body diagonal (the [111] direction). [Figure from Williams and Dunlop, 1995.] Figure 4.1 of Tauxe (2010)</i>)
    </em>
  </figcaption>
</figure>

### Crystallographic directions and Miller indices

Crystallographic **directions** are described using **Miller indices** written in square brackets, **[uvw]**, where the integers *u*, *v*, and *w* give the direction’s components along the crystallographic x-, y-, and z-axes.

- **[100]**, **[010]**, and **[001]** correspond to directions parallel to the cube edges (the crystallographic axes themselves).
- **[110]** lies in the x–y plane and points along a face diagonal of the cube.
- **[111]** points along a body diagonal of the cube.
- A bar over an index (e.g., **[1̅11]**) indicates a negative component along that axis.


### 3D visualization of a magnetic cube

The cube in the visualization represents the cubic crystallographic framework of magnetite. Vectors drawn from the center of the cube point along selected crystallographic **axes and directions**, labeled using Miller indices. These labeled directions illustrate how crystallographic axes, face diagonals, and body diagonals are defined in a cubic crystal.

In [ ]:
# @title Magnetite Crystal Visualization { display-mode: "form" }

import plotly.graph_objects as go
import numpy as np
import itertools

def format_miller(h, k, l):
    """
    Converts indices to standard crystallographic string with overbars.
    Example: (-1, 1, -1) -> "[1̅11̅]"
    """
    def to_char(n):
        if n < 0:
            return f"{abs(n)}\u0305" # Unicode combining overline
        return str(n)

    return f"[{to_char(h)}{to_char(k)}{to_char(l)}]"

def plot_crystal_habit(show_plane=False):
    """
    Visualizes the Magnetite crystal habit.

    Parameters:
    -----------
    show_plane : bool
        If True, visualizes the (1-10) rotation plane (Cyan).
        Default is False.
    """

    # 1. Define Crystal Shape (Cube)
    v_val = 0.75
    x_cube = [v_val, v_val, -v_val, -v_val, v_val, v_val, -v_val, -v_val]
    y_cube = [v_val, -v_val, -v_val, v_val, v_val, -v_val, -v_val, v_val]
    z_cube = [v_val, v_val, v_val, v_val, -v_val, -v_val, -v_val, -v_val]

    i_ind = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2]
    j_ind = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3]
    k_ind = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6]

    fig = go.Figure()

    # TRACE 1: The Cube
    fig.add_trace(go.Mesh3d(
        x=x_cube, y=y_cube, z=z_cube,
        i=i_ind, j=j_ind, k=k_ind,
        color='#A0A0A0',
        opacity=1.0,
        flatshading=True,
        lighting=dict(ambient=0.6, diffuse=0.9, roughness=0.1, specular=0.5),
        hoverinfo='name', name='Magnetite Crystal'
    ))

    # TRACE 2: The (1-10) Plane (Optional)
    if show_plane:
        d = 1.6
        fig.add_trace(go.Mesh3d(
            x=[d, -d, -d, d],
            y=[d, -d, -d, d],
            z=[d, d, -d, -d],
            i=[0, 0], j=[1, 2], k=[2, 3],
            color='cyan', opacity=0.3,
            hoverinfo='name', name='(11̅0) Plane'
        ))

    # FACE NORMALS <100> (Red Hard Axes)
    hard_axes = [
        ([1,0,0]), ([-1,0,0]),
        ([0,1,0]), ([0,-1,0]),
        ([0,0,1]), ([0,0,-1])
    ]

    for vec in hard_axes:
        v = np.array(vec) * 1.8
        label_text = format_miller(*vec)

        fig.add_trace(go.Scatter3d(
            x=[0, v[0]], y=[0, v[1]], z=[0, v[2]],
            mode='lines', line=dict(color='red', width=5),
            hoverinfo='skip', showlegend=False
        ))
        fig.add_trace(go.Scatter3d(
            x=[v[0]], y=[v[1]], z=[v[2]],
            mode='text', text=[f"<b>{label_text}</b>"],
            textfont=dict(color='red', size=14, family="Arial"),
            hoverinfo='skip', showlegend=False
        ))

    # BODY DIAGONALS <111> (Blue Easy Axes)
    for x, y, z in itertools.product([1, -1], repeat=3):
        vec = np.array([x, y, z])
        norm = np.linalg.norm(vec)
        v = (vec / norm) * 1.8

        label_text = format_miller(x, y, z)

        fig.add_trace(go.Scatter3d(
            x=[0, v[0]], y=[0, v[1]], z=[0, v[2]],
            mode='lines', line=dict(color='blue', width=4, dash='dash'),
            hoverinfo='skip', showlegend=False
        ))
        fig.add_trace(go.Scatter3d(
            x=[v[0]], y=[v[1]], z=[v[2]],
            mode='text', text=[f"<b>{label_text}</b>"],
            textfont=dict(color='blue', size=12, family="Arial"),
            hoverinfo='skip', showlegend=False
        ))

    fig.update_layout(
        width=600, height=500,
        margin=dict(r=10, b=10, l=10, t=50),
        title=dict(text='<b>Crystallographic axes of a magnetite cube</b>', x=0.5, y=0.95),
        hovermode=False,
        scene=dict(
            xaxis=dict(visible=False, showspikes=False),
            yaxis=dict(visible=False, showspikes=False),
            zaxis=dict(visible=False, showspikes=False),
            aspectmode='data',
            camera=dict(eye=dict(x=1., y=-1., z=0.6)),
            dragmode='orbit'
        ),
    )

    fig.show()

plot_crystal_habit(show_plane=False)

## Magnetocrystalline Anisotropy

Magnetite ($Fe_3O_4$) has a cubic inverse spinel crystal structure. For **equant** grains (where shape anisotropy is negligible), the magnetic behavior is dominated by magnetocrystalline anisotropy.

Consequently, equant magnetite is **not** uniaxial and does not possess a single easy axis of magnetization. Instead, the lowest-energy magnetization directions correspond to the four body diagonals of the cube, i.e., the [111] crystallographic directions.

For such a cubic crystal, the magnetocrystalline anisotropy energy density is given by **Equation 4.2** in the Essentials of Paleomagnetism text:

$$
\epsilon_a = K_1(\alpha_1^2 \alpha_2^2 + \alpha_2^2 \alpha_3^2 + \alpha_3^2 \alpha_1^2)
+ K_2 \alpha_1^2 \alpha_2^2 \alpha_3^2
$$

where:
* $K_1$ and $K_2$ are the first- and second-order cubic anisotropy constants.
* For magnetite at room temperature, $K_1 = -1.35 \times 10^4$ J m$^{-3}$ and
  $K_2 = -0.44 \times 10^4$ J m$^{-3}$.
* $\alpha_1$, $\alpha_2$, and $\alpha_3$ are the direction cosines of the magnetization vector relative to the crystallographic $x$, $y$, and $z$ axes.

### Getting started with calculations

In the code cell below, we first define the radius of a single-domain magnetite particle (22 nm) and convert it to meters. Using this radius, we compute the volume of the particle assuming a spherical geometry. The anisotropy constants $K_1$ and $K_2$ are then specified numerically, and the particle volume and constants are printed. This sets up the physical parameters needed to later convert the anisotropy *energy density* $\epsilon_a$ into a total anisotropy energy by multiplying by the particle volume.

In [ ]:
radius_nm = 22
r_meters = radius_nm * 1e-9

K1 = -1.35e4
K2 = -0.44e4

# Volume of the spherical particle
V = (4/3) * np.pi * r_meters**3

print(f"Particle Volume: {V:.2e} m^3")
print(f"Anisotropy Constants: K1={K1}, K2={K2}")

## Define the Rotation Path

The magnetocrystalline anisotropy energy depends on the 3D orientation of the magnetization vector. To visualize this on a 2D plot, we choose a specific crystallographic rotation path.

Let's consider the **(11̅0) plane**, which intersects the key symmetry directions of a cubic crystal:

In [ ]:
plot_crystal_habit(show_plane=True)

### Calculating the Anisotropy Energy Profile

To visualize the energy barriers, we simulate rotating the magnetization vector along a specific path through the crystal: the **$(1\bar{1}0)$ plane**. The code performs four key steps to generate the plot:

1.  **Defining the Path (Geometry):**
    We define an angle $\theta$ that rotates from $0^\circ$ (the [001] axis) to $180^\circ$. To constrain the rotation to the $(1\bar{1}0)$ diagonal slice, we set the direction cosines such that $\alpha_1 = \alpha_2$. The vertical component is defined by $\alpha_3 = \cos\theta$, while the horizontal component is split equally ($\alpha_1 = \alpha_2 = \frac{1}{\sqrt{2}}\sin\theta$) to ensure the vector remains normalized to a length of 1.

2.  **Calculating Energy Density (Physics):**
    We apply the cubic anisotropy equation (Tauxe Eq 4.2). This sums two terms:
    $$E_{density} = K_1(\alpha_1^2\alpha_2^2 + \alpha_2^2\alpha_3^2 + \alpha_3^2\alpha_1^2) + K_2(\alpha_1^2\alpha_2^2\alpha_3^2)$$
    * **Term 1 ($K_1$):** Controls the primary shape (Hard vs Easy axes).
    * **Term 2 ($K_2$):** Adds fine structural details (creates the "dimples" or valleys along the diagonals).

3.  **Scaling to the Particle (Volume):**
    The formula above gives Energy Density (Joules/$m^3$). To get the total energy cost for the specific grain, we multiply by the particle's volume ($V = \frac{4}{3}\pi r^3$).

4.  **Normalization (Relative Energy):**
    We are interested in the *barrier height* (the change in energy), not the absolute value. The code subtracts the minimum energy value (`E_raw - min`) so that the most stable state (the Easy Axis) sits at 0 Joules.

Key orientations along this path include the **Hard Axis** [001] at $0^\circ$, the **Easy Axis** [111] at $\approx 55^\circ$, and the **Intermediate Axis** [110] at $90^\circ$. The plot below visualizes this energy landscape.

Let's define a function to use for this calculation:

In [ ]:
def plot_anisotropy_energy(K1, K2, radius_nm=22, ax=None, color='#4A90E2', label=None, linestyle='-'):
    """
    Calculates and plots the anisotropy energy profile for rotation in the (1-10) plane.

    Parameters:
    -----------
    K1, K2 : float
        Anisotropy constants (J/m^3).
    radius_nm : float
        Particle radius in nm.
    ax : matplotlib.axes.Axes, optional
        The axis to plot on. If None, uses current axis.
    """
    if ax is None:
        ax = plt.gca()

    # A. GEOMETRY: Rotation from [001] to [00-1] via [110]
    theta_deg = np.linspace(0, 180, 500)
    theta_rad = np.radians(theta_deg)

    # Direction Cosines for (1-10) plane
    a1 = (1/np.sqrt(2)) * np.sin(theta_rad)
    a2 = a1
    a3 = np.cos(theta_rad)

    # B. PHYSICS: Calculate Cubic Energy (Eq 4.2 of Tauxe, 2010)
    term1 = (a1**2 * a2**2) + (a2**2 * a3**2) + (a3**2 * a1**2)
    term2 = a1**2 * a2**2 * a3**2

    volume = (4/3) * np.pi * (radius_nm * 1e-9)**3
    E_raw = ((K1 * term1) + (K2 * term2)) * volume

    # C. NORMALIZE: Shift so the Minimum Energy (Easy Axis) is 0
    E_plot = E_raw - np.min(E_raw)

    # D. PLOT
    ax.plot(theta_deg, E_plot, color=color, linewidth=3, label=label, linestyle=linestyle)
    ticks = [0, 54.7, 90, 125.3, 180]
    tick_labels = ['[001]\nHard', '[111]\nEasy', '[110]\nMedium', '[11\u03051]\nEasy', '[001\u0305]\nHard']

    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_xlim(0, 180)
    ax.set_xlabel("Rotation Angle (Degrees)", fontsize=12)
    ax.set_ylabel("Anisotropy Energy (Joules)", fontsize=12)
    ax.grid(True, alpha=0.3)

    return E_plot

Let's now put the function to use:

In [ ]:

plt.figure(figsize=(8, 5))
plot_anisotropy_energy(K1=-1.35e4, K2=-0.44e4, radius_nm=22)
plt.title("Energy Landscape of Cubic Magnetite", fontsize=14)
plt.tight_layout()
plt.show()

### Interpreting the Energy Landscape

This plot visualizes the energy "cost" required to rotate the magnetization vector through the crystal lattice. It reveals the magnetic personality of magnetite:

* **The Valleys ([111]):** These global minima are the **Easy Axes**. Just as a ball rolls to the bottom of a hill, the magnetization vector naturally aligns along these body diagonals. These are the stable "resting states" for the magnetic moment.
* **The Peaks ([001]):** These are the **Hard Axes**. Aligning the magnetization with the cube edges requires the maximum input of energy, making these the most unstable configurations.
* **The Barrier ([110]):** The intermediate peak represents the "pass" or saddle point between valleys.

**Key Takeaway:** The stability of a magnetic recording (**remanence**) is defined by these hills. To flip the magnetization from one stable [111] direction to another, an external force (like a magnetic field) or thermal fluctuation must push the moment over the [110] energy barrier.

In [ ]:
# @title Magnetite Anisotropy Visualization { display-mode: "form" }

import plotly.graph_objects as go
import numpy as np
import itertools

def format_miller(h, k, l):
    """Converts indices to standard crystallographic string with overbars."""
    def to_char(n):
        if n < 0:
            return f"{abs(n)}\u0305"
        return str(n)
    return f"[{to_char(h)}{to_char(k)}{to_char(l)}]"

def plot_magnetite_anisotropy(K1=-1.35e4, K2=-0.44e4, radius_nm=25, exaggeration=0.6, cube_scale=1.0, global_max=None):
    """
    Visualizes anisotropy with locked scaling for true comparisons.

    Parameters:
    -----------
    global_max : float, optional
        The maximum energy value (Joules) to use for scaling BOTH the
        color bar and the geometric distortion.
        Use the max energy from Room Temp (approx 2.0e-19) to see
        how flat the landscape becomes at 120 K.
    """

    # --- 1. MATH: Energy Surface ---
    phi = np.linspace(0, 2 * np.pi, 100)
    theta = np.linspace(0, np.pi, 100)
    phi, theta = np.meshgrid(phi, theta)

    # Direction Cosines
    a1 = np.sin(theta) * np.cos(phi)
    a2 = np.sin(theta) * np.sin(phi)
    a3 = np.cos(theta)

    # Energy Density Calculation
    term1 = (a1**2 * a2**2) + (a2**2 * a3**2) + (a3**2 * a1**2)
    term2 = a1**2 * a2**2 * a3**2
    E_density = (K1 * term1) + (K2 * term2)

    # Convert to Total Energy (J) and Normalize
    r_particle = radius_nm * 1e-9
    Volume = (4/3) * np.pi * r_particle**3
    E_total = E_density * Volume
    E_norm = E_total - E_total.min()

    # --- SCALING LOGIC ---
    # If global_max is not set, scale to the current data (local max)
    if global_max is None:
        scale_denom = E_norm.max()
        cmax_val = None # Let Plotly auto-scale color
    else:
        scale_denom = global_max
        cmax_val = global_max

    # Map energy to radius (Geometric Scaling)
    # We use scale_denom to ensure the shape distortion is proportional to the global max
    r_blob = 1.0 + (exaggeration * (E_norm / scale_denom))

    x_blob = r_blob * np.sin(theta) * np.cos(phi)
    y_blob = r_blob * np.sin(theta) * np.sin(phi)
    z_blob = r_blob * np.cos(theta)

    # --- 2. GEOMETRY: The Reference Cube ---
    v_val = 0.6 * cube_scale
    x_cube = [v_val, v_val, -v_val, -v_val, v_val, v_val, -v_val, -v_val]
    y_cube = [v_val, -v_val, -v_val, v_val, v_val, -v_val, -v_val, v_val]
    z_cube = [v_val, v_val, v_val, v_val, -v_val, -v_val, -v_val, -v_val]

    i_ind = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2]
    j_ind = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3]
    k_ind = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6]

    # --- 3. PLOTTING ---
    fig = go.Figure()

    # TRACE 0: Energy Surface
    fig.add_trace(go.Surface(
        z=z_blob, x=x_blob, y=y_blob,
        surfacecolor=E_norm,
        cmin=0,
        cmax=cmax_val,        # Locks Color Scale
        colorscale='magma_r',
        colorbar=dict(title='Energy Barrier (J)', len=0.5, thickness=15, x=0.9, exponentformat='e'),
        opacity=1.0,
        hoverinfo='none',
        contours_x=dict(highlight=False), contours_y=dict(highlight=False), contours_z=dict(highlight=False),
        name='Energy'
    ))

    # TRACE 1: Reference Cube
    fig.add_trace(go.Mesh3d(
        x=x_cube, y=y_cube, z=z_cube, i=i_ind, j=j_ind, k=k_ind,
        color='silver', opacity=1, flatshading=True, lighting=dict(ambient=0.5, diffuse=0.8),
        hoverinfo='skip', visible=False, name='Cube'
    ))

    # --- 4. AXIS GENERATION ---
    max_extent = max(1.0 + exaggeration, v_val)
    axis_scale = max_extent + 0.5

    # Standard Axis Code (Condensed)
    hard_axes = [([1,0,0]), ([-1,0,0]), ([0,1,0]), ([0,-1,0]), ([0,0,1]), ([0,0,-1])]
    for vec in hard_axes:
        v = np.array(vec) * axis_scale
        fig.add_trace(go.Scatter3d(x=[0, v[0]], y=[0, v[1]], z=[0, v[2]], mode='lines', line=dict(color='red', width=5), hoverinfo='skip', showlegend=False))
        fig.add_trace(go.Scatter3d(x=[v[0]], y=[v[1]], z=[v[2]], mode='text', text=[f"<b>{format_miller(*vec)}</b>"], textfont=dict(color='red', size=12), hoverinfo='skip', showlegend=False))

    for x, y, z in itertools.product([1, -1], repeat=3):
        vec = np.array([x, y, z])
        v = (vec / np.linalg.norm(vec)) * axis_scale
        fig.add_trace(go.Scatter3d(x=[0, v[0]], y=[0, v[1]], z=[0, v[2]], mode='lines', line=dict(color='blue', width=4, dash='dash'), hoverinfo='skip', showlegend=False))
        fig.add_trace(go.Scatter3d(x=[v[0]], y=[v[1]], z=[v[2]], mode='text', text=[f"<b>{format_miller(x,y,z)}</b>"], textfont=dict(color='blue', size=11), hoverinfo='skip', showlegend=False))

    # --- 5. LAYOUT ---
    n_traces = len(fig.data)
    vis_energy = [True, False] + [True] * (n_traces - 2)
    vis_cube   = [False, True] + [True] * (n_traces - 2)

    fig.update_layout(
        width=800, height=700,
        margin=dict(r=10, b=10, l=10, t=50),
        title=dict(text='<b>Magnetite Anisotropy Map</b>', x=0.5, y=0.95),
        hovermode=False,
        updatemenus=[dict(
            type="buttons", direction="down", x=0.02, y=0.98, bgcolor="rgba(255, 255, 255, 0.9)",
            buttons=list([
                dict(label="Energy Landscape", method="update",
                     args=[{"visible": vis_energy}, {"title": "<b>Magnetite Energy Surface</b>"}]),
                dict(label="Crystal Habit", method="update",
                     args=[{"visible": vis_cube}, {"title": "<b>Physical Crystal Shape</b>"}]),
            ]),
        )],
        scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False),
            aspectmode='data', camera=dict(eye=dict(x=1., y=1., z=1.)), dragmode='orbit'),
    )
    fig.show()

plot_magnetite_anisotropy(K1=-1.35e4, K2=-0.44e4, cube_scale=1.5)

## Anisotropy as a function of temperature

> Because electronic interactions depend heavily on inter-atomic spacing, magnetocrystalline anisotropy constants are a strong function of temperature (see Figure 4.2). In magnetite, $K_1$ changes sign at a temperature known as the isotropic point. At the isotropic point, there is no large magnetocrystalline anisotropy. The large energy barriers that act to keep the magnetizations parallel to the body diagonal are gone, and the spins can wander more freely through the crystal. *Tauxe (2010)*

<figure>
  <img src="https://raw.githubusercontent.com/Institute-for-Rock-Magnetism/2026_ESCI_pmag_course/main/W3_anisotropy_hysteresis/images/anisotropy_magnetite.png" width="500px">
  <figcaption>
    <em>
      Variation of K$_1$ and K$_2$ of magnetite as a function of temperature. [Solid lines are data from
Syono and Ishikawa (1963). Dashed lines are data from Fletcher and O'Reilly (1974).] Figure 4.2 of Tauxe (2010)</i>)
    </em>
  </figcaption>
</figure>

> ### 🔎 Temperature Reminder: What is Kelvin?
>
> **Kelvin (K)** is the SI unit of temperature used in physics and thermodynamics.  
> Unlike Celsius, it is an **absolute temperature scale**: **0 K** corresponds to absolute zero, where thermal motion is minimized.
>
> **Conversion between scales:**
>
> $T(\mathrm{K}) = T(^{\circ}\mathrm{C}) + 273.15$;
>
> $ T(^{\circ}\mathrm{C}) = T(\mathrm{K}) - 273.15$
>
> **Room temperature** is approximately:
>
> $20\text{–}25^{\circ}\mathrm{C} \;\approx\; 293\text{–}298\ \mathrm{K}$,
>
> and is commonly rounded to $T \approx 300\ \mathrm{K}$ for back-of-the-envelope calculations.

### **Exercise: Investigate the Isotropic Point**

We have analyzed the energy landscape at room temperature (300 K). Now, it is your turn to model what happens at the **Isotropic Point**.

1.  **Analyze the Chart:** Look back at the variation of K$_1$ and K$_2$ of magnetite as a function of temperature above.
2.  **Find the Values:** Locate the temperature where $K_1$ crosses zero ($\approx 130$ K). What is the value of $K_2$ at this temperature?
3.  **Update the Code:** Enter your estimates for `K1` and `K2` in the cell below in the second call to `plot_anisotropy_energy` to generate the comparison.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

# Room Temp (300 K)
plot_anisotropy_energy(K1=-1.35e4, K2=-0.44e4, radius_nm=22, ax=ax,
                       color='darkred', label='300 K (Room Temp)')

# Enter K1 and K2 at the isotropic Point (~130 K)
plot_anisotropy_energy(K1= , K2= , radius_nm=22, ax=ax,
                       color='darkblue', linestyle='-.',
                       label='130 K (Isotropic Point)')

ax.set_title("Direct Comparison: 300 K vs 130 K Barrier", fontsize=14)
ax.legend(fontsize=12)
plt.tight_layout()
plt.show()

### **Exercise: Investigate the Isotropic Point**

Now model what happens at the **Isotropic Point** as visualized in 3D. Use the `plot_magnetite_anisotropy()` function and input the `K1` and `K2` values as above. Also, go to the previous plot to find the maximum of the color bar. Set that as the `global_max`. Experiment with plotting with and without setting the `global_max`. Without it set, the plot will autoscale and accentuate the slight differences in anisotropy due to K2 being non-zero.

## Shape Anisotropy

We have seen how the internal lattice structure creates preferred magnetic directions (magnetocrystalline anisotropy). But there is another powerful competitor in the arena: the **physical shape** of the grain.

For strongly magnetic minerals like magnetite, the grain's shape often matters more than the crystal structure. This is called **Shape Anisotropy**.

#### The Easy Long Axis
Imagine a bar magnet. It generates a magnetic field around it, but it also creates a field *inside* itself called the **Demagnetizing Field**. This internal field tries to push back against the magnetization.

* **Along the Short Axis:** The magnetic poles on the surface are close together, creating a strong repulsive field. This is a **High Energy** state.
* **Along the Long Axis:** The poles are far apart, creating a weak repulsive field. This is a **Low Energy** state.

Therefore, the magnetization always "wants" to align with the long axis of the grain to minimize energy.

The energy cost to rotate away from this long axis follows a straightforward uniaxial trend. If $\theta$ is the angle away from the long axis:

$$
E_{shape} = K_{shape} \sin^2 \theta
$$

The strength of this barrier ($K_{shape}$) depends on the square of the magnetization ($M_s^2$) and the difference in the grain's dimensions ($\Delta N$):

$$
K_{shape} = \frac{1}{2} \mu_o M_s^2 \Delta N
$$

Because the magnetization ($M_s$) for magnetite is huge, even a tiny value for $\Delta N$ (a tiny elongation) can create a massive energy barrier.

### Interactively explore the shape takeover

The interactive tool below compares the magnetocrystalline anisotropy (the grey dashed line) against the shape anisotropy (the red curve).

1.  **Start at 1.0:** A perfect sphere has zero shape anisotropy. The crystal lattice is in control.
2.  **Drag the Slider:** Slowly increase the aspect ratio to elongate the grain.
3.  **Find the Tipping Point:** Explore the slider to find the aspect ratio where the Shape Energy barrier (Red) becomes larger than the Crystal Energy barrier (Grey).

*Does the grain have to be a long needle for shape to win, or does it happen sooner than you expect?*

In [ ]:
# Physics Constants
Ms = 480e3       # Magnetite Saturation Magnetization (A/m)
K1 = -1.35e4     # First Order Crystal Anisotropy (J/m^3)
K2 = -0.44e4     # Second Order Crystal Anisotropy (J/m^3)
radius_nm = 22   # Radius of equivalent sphere
mu_0 = 4 * np.pi * 1e-7

# Calculated Constants
volume = (4/3) * np.pi * (radius_nm * 1e-9)**3
barrier_density = abs(K1/3 + K2/27)
E_crystal_barrier = barrier_density * volume

def calculate_demag_factors(q):
    """
    Calculates demagnetizing factors for a prolate ellipsoid.

    Args:
        q (float): Aspect ratio (long axis / short axis).

    Returns:
        tuple: (Na, Nb) where Na is the factor along the long axis
               and Nb is the factor along the short axis.
    """
    if q <= 1.0001:
        return 1/3, 1/3

    e = np.sqrt(1 - (1/q)**2)
    Na = ((1 - e**2) / (2 * e**3)) * (np.log((1+e)/(1-e)) - (2*e))
    Nb = (1 - Na) / 2
    return Na, Nb

def update_view(aspect_ratio=1.0):
    """
    Updates the visualization of the grain geometry and the corresponding
    energy landscape based on the provided aspect ratio.
    """
    fig, (ax_shape, ax_energy) = plt.subplots(1, 2, figsize=(12, 5))

    # --- Visualization Panel ---
    b = radius_nm * (1/aspect_ratio)**(1/3)
    a = b * aspect_ratio

    ellipse = Ellipse((0, 0), width=b*2, height=a*2, color='#34495E', alpha=0.8)
    ax_shape.add_patch(ellipse)
    ax_shape.set_xlim(-45, 45)
    ax_shape.set_ylim(-45, 45)
    ax_shape.set_aspect('equal')
    ax_shape.axis('off')
    ax_shape.set_title(f"Grain Geometry\nAspect Ratio {aspect_ratio:.2f}:1", fontsize=12)

    ax_shape.annotate('', xy=(20, 0), xytext=(20, a), arrowprops=dict(arrowstyle='<->', color='red'))
    ax_shape.text(25, a/2, "Easy Axis", color='red', rotation=90, va='center')

    # --- Energy Panel ---
    theta_deg = np.linspace(0, 90, 100)

    Na, Nb = calculate_demag_factors(aspect_ratio)
    K_shape_joules = 0.5 * mu_0 * volume * Ms**2 * (Nb - Na)
    E_shape = K_shape_joules * np.sin(np.radians(theta_deg))**2

    ax_energy.axhline(E_crystal_barrier, color='gray', linestyle='--', linewidth=2, label='Max Crystal Barrier')
    ax_energy.plot(theta_deg, E_shape, color='#E74C3C', linewidth=3, label='Shape Energy Barrier')

    ax_energy.set_title("Competition: Shape vs. Crystal", fontsize=12)
    ax_energy.set_ylabel("Energy (Joules)", fontsize=11)
    ax_energy.set_xlabel("Rotation Angle from Long Axis (°)", fontsize=11)
    ax_energy.set_xlim(0, 90)
    ax_energy.set_ylim(0, E_crystal_barrier * 5)
    ax_energy.grid(True, alpha=0.3)
    ax_energy.legend(loc='upper right')

    max_shape = np.max(E_shape)
    ratio = max_shape / E_crystal_barrier if E_crystal_barrier > 0 else 0

    stats_text = (f"Shape Barrier: {max_shape:.1e} J\n"
                  f"Crystal Barrier: {E_crystal_barrier:.1e} J\n"
                  f"Dominance: {ratio:.1f}x")

    box_color = '#FADBD8' if ratio > 1.0 else '#EAEDED'
    ax_energy.text(5, E_crystal_barrier * 4, stats_text,
                   bbox=dict(facecolor=box_color, alpha=0.9), fontsize=10)

    if ratio > 1.0:
        ax_energy.text(45, E_crystal_barrier * 1.2, "SHAPE WINS",
                       color='#C0392B', fontweight='bold', ha='center')

    plt.tight_layout()
    plt.show()

# Widget Setup
style = {'description_width': 'initial'}
slider = widgets.FloatSlider(
    value=1.0,
    min=1.0,
    max=3.0,
    step=0.05,
    description='Aspect Ratio (Long/Short):',
    style=style,
    layout=widgets.Layout(width='500px')
)

widgets.interactive(update_view, aspect_ratio=slider)

### Interactively explore the shape takeover

The interactive tool below compares the magnetocrystalline anisotropy (the grey dashed line) against the shape anisotropy (the red curve).

1.  **Start at 1.0:** A perfect sphere has zero shape anisotropy. The crystal lattice is in control.
2.  **Drag the Slider:** Slowly increase the aspect ratio to elongate the grain.
3.  **Find the Tipping Point:** Explore the slider to find the aspect ratio where the Shape Energy barrier (Red) becomes larger than the Crystal Energy barrier (Grey).

*Does the grain have to be a long needle for shape to win, or does it happen sooner than you expect?*

### The Battle for Magnetic Memory: Thermal Stability

How can a microscopic mineral grain hold a magnetic recording for billions of years?

To answer this, we have to look at the "tug-of-war" happening at the atomic scale.
1.  **Anisotropy Energy ($E_a$):** This is the energy barrier that locks the magnetization in place (caused by the crystal shape or lattice structure). It is the "hill" that the magnetization must climb to flip direction.
2.  **Thermal Energy ($E_T$):** This is the chaotic vibration of atoms caused by heat. It constantly jostles the magnetic moment, trying to randomize it.

If the thermal energy is high enough, it can kick the magnetization over the barrier, causing the grain to lose its memory.

#### Néel relaxation theory

The time it takes for a grain to randomly flip due to heat is called the **Relaxation Time ($\tau$)**. It is governed by statistical mechanics using the Arrhenius relationship:

<p class="math">
\[
\tau
= \frac{1}{C}
\exp\!\left(
\frac{\text{anisotropy energy}}{\text{thermal energy}}
\right)
= \frac{1}{C}
\exp\!\left(
\frac{K V}{k_B T}
\right)
\]
</p>

<p class="math">
\[
\text{Since } \exp(x) = e^{x}, \text{ this can also be written as}
\]
</p>

<p class="math">
\[
\tau
= \frac{1}{C}
e^{\left(\frac{K V}{k_B T}\right)}
\]
</p>

#### Breaking Down the Variables

* **$\tau$ (Relaxation Time):** The average time the magnetization stays in one direction before flipping.
* **$C$ (Frequency Factor):** This represents the "attempt rate"—how often the magnetic moment tries to climb the hill. It is related to atomic vibration frequencies, typically estimated at $10^{10}$ Hz ($10^{10}$ attempts per second).
* **$K$ (Anisotropy Energy Density):** This represents the **height of the energy barrier per unit volume**. It defines how energetically "expensive" it is to rotate the magnetization away from its preferred direction.
    * For spherical grains, this is the **Crystalline Anisotropy** ($K_1$).
    * For elongate grains, this is dominated by the **Shape Anisotropy** ($K_{shape}$).
* **$v$ (Volume):** The size of the grain.
* **$kT$ (Thermal Energy):** The Boltzmann constant ($k$) times Temperature ($T$).

#### The "Exponential" Power
The most critical part of this equation is that the Volume ($v$) is **inside the exponent**.

This means that $\tau$ does not change linearly with size. It changes exponentially. A grain that is slightly too small might flip every nanosecond. A grain that is just a *tiny bit* bigger might take billions of years to flip.


### Example: Stability of Perfect Spheres

Before you tackle the assignment (elongated grains), let's look at the baseline: **Equant Grains** (perfect spheres).

For a sphere, there is no shape anisotropy ($N_a = N_b$). The only thing holding the magnetization in place is the crystal lattice itself (**Magnetocrystalline Anisotropy**). The barrier density is determined by the crystal constant $K_1$.

The code below calculates the relaxation time for spherical Magnetite. Run this to see the "Stability Cliff"—notice how a tiny increase in diameter takes a grain from unstable (forgetting in seconds) to stable (lasting billions of years).

*This treatment assumes uniformly magnetized single-domain grains; larger grains will adopt vortex and then multidomain states that introducing additional energy minima and reversal mechanisms that invalidate the simple Néel relaxation framework used here.*

In [ ]:
# --- 1. SET CONSTANTS ---
kB = 1.38e-23           # Boltzmann Constant (J/K)
T = 300                 # Temperature (Kelvin)
C = 1e10                # Frequency factor (Hz) - atomic vibration speed

# --- Physical Constants (SI Units) ---
# Aligned with Tauxe Appendix A
k_B = 1.380649e-23      # Boltzmann constant (J/K)
mu_0 = 4 * np.pi * 1e-7 # Permeability of free space (H/m)
tau_0 = 1e-10             # Frequency factor (seconds)

# Magnetite Properties
K1 = -1.35e4            # Crystal Anisotropy (J/m^3)
# The effective barrier to flip is roughly 1/12th of K1 due to saddle points
K_eff = abs(K1) / 12

# --- 2. CALCULATE RELAXATION TIME ---
# We look at grain diameters from 10 nm to 80 nm
diameters_nm = np.linspace(10, 80, 40)
diameters_m = diameters_nm * 1e-9

# Step A: Calculate Volume (Sphere)
radius_m = diameters_m / 2
volume = (4/3) * np.pi * (radius_m)**3

# Step B: Calculate Energy Barrier (K * V)
energy_barrier = K_eff * volume

# Step C: Arrhenius Equation
thermal_energy = kB * T
tau = (1/C) * np.exp(energy_barrier / thermal_energy)

# --- 3. PLOT ---
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(diameters_nm, tau, linewidth=3, color='#2980B9')
ax.axhline(100, color='green', linestyle='--', label='100 Seconds (Lab Time)')
ax.text(21, 150, "Stable (barely) in the Lab", color='green')
ax.axhline(4.54e9 * 365.25 * 24 * 60 * 60, color='red', linestyle='--', label='Age of Earth')
ax.text(21, 1e17, "Stable for Geologic Time", color='red')
ax.set_yscale('log') # Use Log scale because time varies wildly!
ax.set_xlabel('Grain Diameter (nm)', fontsize=12)
ax.set_ylabel('Relaxation Time (seconds)', fontsize=12)
ax.set_title('Stability of Spherical Magnetite Grains', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_ylim(1e-9, 1e20) # Limit y-axis to readable range

plt.tight_layout()
plt.show()

# **Assignment: The Billion-Year Hard Drive**

### **The Challenge**
Small magnetic grains lose their memory because thermal energy constantly tries to flip their magnetization (a process described by **Néel Theory**). However, as grains get slightly larger, they become incredibly stable.

Your task is twofold:
1.  Determine the **exact grain width** (short axis $a$) required for a magnetite grain with an elongated aspect ratio of 1.5 to remain stable for **1 Billion Years**.
2.  Calculate how hot you would have to heat this specific grain to wipe its memory in just **60 seconds** (a typical laboratory experiment).

### **The Parameters**
* **Geometry:** Rectangular Prism (square cross-section $a \times a$, length $b$)
* **Aspect Ratio ($q$):** 1.5 (where $q = b/a$)
* **Frequency Factor ($C$):** $1 \times 10^{10}$ Hz
* **Magnetization ($M_s$):** $480 \times 10^3$ A/m
* **Boltzmann Constant ($k_B$):** $1.38 \times 10^{-23}$ J/K
* **Target Time 1 ($\tau_{geo}$):** $1 \times 10^9$ years (**Note:** Convert this to seconds!)
* **Target Time 2 ($\tau_{lab}$):** 60 seconds

---

### **Python Math Cheatsheet**
You will need the `numpy` library (usually imported as `np`).

* **Natural Log ($\ln x$):** Use `np.log(x)`.
* **Exponential ($e^x$):** Use `np.exp(x)`.
* **Powers ($x^3$ or $\sqrt[3]{x}$):** Use the `**` operator.
    * $x^2$ $\rightarrow$ `x**2`
    * $\sqrt[]{x}$ $\rightarrow$ `x**(1/2)`
    * $\sqrt[3]{x}$ $\rightarrow$ `x**(1/3)`

---

### **The Recipe**

You will need to write a Python code in which you rearrange the **Néel relaxation equation** to do the analysis:
$$\tau = \frac{1}{C} \exp\left( \frac{K V}{k_B T} \right)$$

#### **Step 1: Calculate the Anisotropy Energy Density ($K$)**
First, we calculate the energy barrier per unit volume. Tauxe states that for elongate particles dominated by shape, the microscopic coercivity ($H_k$) simplifies to $\Delta N M_s$.

1.  **Find $\Delta N$:** Use the function `calculate_demag_factors(1.5)` from the widget code provided in class.
2.  **Calculate Coercivity ($H_k$):** $H_k = \Delta N \cdot M_s$
3.  **Calculate Energy Density ($K$):** Convert the field strength ($H_k$) to energy density (Joules/m³): $K = \frac{1}{2} \mu_0 M_s H_k$

#### **Step 2: Solve for Critical Volume ($V_{crit}$)**
Using the **Néel relaxation equation** (top of this section), rearrange the variables to solve for **Volume ($V$)**.
* *Hint:* You need to isolate $V$. This will involve taking the natural log of both sides.
* Calculate the critical volume required for $\tau_{geo}$ at $T = 300$ K.

#### **Step 3: Convert Volume to Width**
We are modeling the grain as a **Rectangular Prism** with width $a$ and length $b = 1.5a$.
1.  Write the equation for the Volume of this prism in terms of width $a$.
2.  Invert that equation to solve for $a$.
3.  Calculate the width in meters, then convert to nanometers.

#### **Step 4: The Unblocking Temperature ($T_b$)**
Now, take that *exact same grain* (using the volume $V_{crit}$ you just found) and imagine we heat it up in the lab. We want to find the Temperature ($T$) where the relaxation time drops to $\tau_{lab} = 60$ seconds.

* Rearrange the **Néel relaxation equation** to solve for **Temperature ($T$)**.
* Calculate the temperature (in Kelvin) required to reduce $\tau$ to 60s.

---

### **Deliverables**

1.  **Generate a Comparison Plot:**
    Create a single plot with **Relaxation Time (y-axis, log scale)** vs. **Grain Width (x-axis, linear)**.
    * **Curve 1:** Plot the curve for $T = 300$ K (Room Temp).
    * **Curve 2:** Plot the curve for $T = T_b$ (Your calculated unblocking temp).
    * **Reference Lines:** Add horizontal lines for **1 Billion Years** and **60 Seconds**.
    * *Self-Check:* If your math is correct, Curve 1 should cross the "1 Billion Year" line at the exact same width that Curve 2 crosses the "60 Second" line!
2.  **Submit to Canvas:**
    * Upload the **Plot image**.
    * Add a **Comment** with your two answers:
        1.  *"Stable Width for 1 Gyr: XX.X nm"*
        2.  *"Unblocking Temperature for this grain (60s scale): XXX K"* **And also provide this temperature in celsius**

### **Starter Code** *If you want it*

Copy and paste this code into a code cell to get started. It contains the helper function and the plotting code you need, but **you must fill in the math** where indicated by the `???`.

```python
import numpy as np
import matplotlib.pyplot as plt

# --- 1. SETUP & CONSTANTS ---
kB = 1.38e-23        # Boltzmann Constant (J/K)
mu_0 = 4 * np.pi * 1e-7
C = 1e10             # Frequency Factor (Hz)
T_room = 300         # Room Temp (K)

# Fills these in based on the assignment parameters:
Ms = ???             # Saturation Magnetization (A/m)
q = ???              # Aspect Ratio
tau_geo_years = ???  # Target time (years)
tau_lab_seconds = 60 # Lab time (seconds)

# Convert years to seconds:
tau_geo_seconds = ???

# --- 2. HELPER FUNCTION ---
# (This calculates the Shape factors Na and Nb for you)
def calculate_demag_factors(ratio):
    e = np.sqrt(1 - (1/ratio)**2)
    Na = ((1 - e**2) / (2 * e**3)) * (np.log((1+e)/(1-e)) - (2*e))
    Nb = (1 - Na) / 2
    return Na, Nb

# --- 3. YOUR CALCULATIONS ---

# [STEP 1] Calculate Anisotropy Energy Density (K)
# Hint: Use the helper function above to get Na, Nb first.
# ...
# ...
K = ???

print(f"Energy Density (K): {K:.2e} J/m^3")

# [STEP 2] Solve for Critical Volume (V_crit)
# Hint: Rearrange the Neel equation.
# ...
V_crit = ???

print(f"Critical Volume: {V_crit:.2e} m^3")

# [STEP 3] Convert Volume to Width (Rectangular Prism)
# Hint: V = 1.5 * a^3. Solve for 'a'.
# ...
width_meters = ???
width_nm = width_meters * 1e9  # Convert to nanometers

print(f"Stable Width for 1 Gyr: {width_nm:.2f} nm")

# [STEP 4] Calculate Unblocking Temperature (Tb)
# Hint: Rearrange Neel equation to solve for T.
# ...
Tb = ???

print(f"Unblocking Temp (Tb) for 60s: {Tb:.2f} K")


# --- 4. VISUALIZATION (Pre-Built for you) ---
# This section generates the plot. You do not need to change the math here,
# just run it to verify your answers from above match the graph.

# A. Create a range of widths to test (from 15nm to 40nm)
# np.linspace(start, stop, number_of_points)
plot_widths_nm = np.linspace(15, 40, 100)
plot_widths_m = plot_widths_nm * 1e-9

# B. Calculate Volume for every width in that list
plot_volumes = 1.5 * plot_widths_m**3

# C. Calculate Relaxation Times
# We use a special function to clip the exponent so the code doesn't crash
# from "Overflow Errors" when the time becomes billions of years.
def get_safe_tau(K_val, T_val):
    exponent = (K_val * plot_volumes) / (kB * T_val)
    exponent = np.clip(exponent, None, 700) # Clip at e^700 (max for Python)
    return (1/C) * np.exp(exponent)

tau_curve_room = get_safe_tau(K, T_room)
tau_curve_high = get_safe_tau(K, Tb)

# D. Plot!
plt.figure(figsize=(10, 6))

# Plot the curves
plt.plot(plot_widths_nm, tau_curve_room, linewidth=2, label=f'Room Temp ({T_room} K)')
plt.plot(plot_widths_nm, tau_curve_high, linewidth=2, label=f'Unblocking Temp ({Tb:.0f} K)')

# Plot the Reference Lines
plt.axhline(tau_geo_seconds, color='gray', linestyle='--', label='1 Billion Years')
plt.axvline(width_nm, color='gray', linestyle=':', label='Your Calc. Width')

plt.axhline(tau_lab_seconds, color='red', linestyle='--', label='60 Seconds')

# Make it readable
plt.yscale('log')
plt.ylim(1e-9, 1e25) # Limits the y-axis so infinities don't squash the plot
plt.xlabel('Grain Width (nm)')
plt.ylabel('Relaxation Time (seconds)')
plt.title(f'Stability of Magnetite (Aspect Ratio {q})')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()